In [ ]:
import os
from netml.pparser.parser import PCAP
import pprint
from scapy.all import *
from scapy.layers.inet import *
from tqdm import tqdm

import tensorflow as tf


In [ ]:
dataDir = "/code/2.Malwares/"
targetFile = "Zeus.pcap"
destination = dataDir + "out"
tsharkPath="/usr/bin/tshark"
packetSizeInBytes = 100
packetsPerFlow = 2
tfrecorsFolder = dataDir + 'tfrecords'

In [ ]:
cats = ["Benign","Malware"]
listofProcessedFiles = []
training_data = []

In [ ]:
def getLabelForFile(filename):
    filename.replace('.pcap','')
    if filename.find('-') != -1:
        filename = filename.split('-')[0]
    for index,processedFile in enumerate(listofProcessedFiles):
        if processedFile == filename:
            return index
    listofProcessedFiles.append(filename)
    return len(listofProcessedFiles)
    
#pprint.pprint(listofProcessedFiles)
#label = getLabelForFile('tes-test')
#print(label)

In [ ]:
def _bytes_feature(value):# peut-être un tableau
    """Returns a bytes_list from a string / byte."""
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy()  # BytesList won't unpack a string from an EagerTensor.
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

In [ ]:
def _int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

In [ ]:
# Create the features dictionary.
def packet_example(packetArray, label):
    feature = {
        'label': _int64_feature(label),
        'packet1': _bytes_feature(packetArray[0]),
        'packet2': _bytes_feature(packetArray[1]),
    }
    return tf.train.Example(features=tf.train.Features(feature=feature))

In [ ]:
 def processPackets(packets)
    newFlow = []
    for i,packet in enumerate(packets):
        packetHex = linehexdump(packet.payload,onlyhex=1,dump=True) # dump packet to hex
        packetHexnoSpace = packetHex.replace(' ','').lower() #to hex
        packetBytes = bytes.fromhex(packetHexnoSpace) # to bytestring
        if len(packetBytes) < packetSizeInBytes: # ajuster les packets à une taille de 100 bytes 
            packetPadded = packetBytes.ljust(packetSizeInBytes,b'\0') #padding
        else:
            packetPadded = packetBytes[:packetSizeInBytes] # couper les paquet trop grand
        if i >= packetsPerFlow:
            #print('je break!')
            break
        newFlow.append(packetPadded)
    return newFlow


In [ ]:
for cat in cats: # pour chaqye dossier Benign, Malware
    path = os.path.join(dataDir,cat)
    for filePcap in os.listdir(path): #parcours les fichiers pour chaque dossier
        pprint.pprint(path)
        pprint.pprint(filePcap)

        parsedPcap = PCAP(
        os.path.join(path,filePcap),
        flow_ptks_thres=packetsPerFlow,
        )
        parsedPcap.pcap2flows(interval=1000)
        for flow in tqdm(parsedPcap.flows): #(val,182.168.1.15,,)(<Ether
            flow = flow[1:] # retirer le premier élément car inutile
            for packets in flow:
                tfrecordsFileName = filePcap.replace('.pcap','.tfrecords')
                with tf.io.TFRecordWriter(tfrecorsFolder + os.sep +tfrecordsFileName) as writer:
                     tf_example = packet_example(processPackets(packets), getLabelForFile(filePcap))
                     writer.write(tf_example.SerializeToString())


"""
TODO : convertir hexdump srting to bytestring (https://www.delftstack.com/howto/python/python-convert-hex-to-byte/)
        - ensure only the two first packet are processed ok
        - implement padding ok
        - label each flow 
        - put everything in pickles
        - tweak to treat whole dataset not juste one pcap file

        [[packet 1,
        packet 2],
        label]
"""





In [ ]:
import random
random.shuffle(training_data)

In [ ]:
X = []
y = []

for features,label in training_data:
    X.append(features)
    y.append(label)



In [ ]:
import pickle

pickle_out = open("X.pickle","wb")
pickle.dump(X, pickle_out)
pickle_out.close()

pickle_out = open("y.pickle","wb")
pickle.dump(y, pickle_out)
pickle_out.close()